### for 7 classes

In [156]:
import re
import pandas as pd

df_train = pd.read_csv('./trainBIO.txt',sep='\t',header=None)
df_train.columns=['token','class']

totalDocs = len(df_train)

df_test = pd.read_csv('./testBIO.txt',sep='\t',header=None)
df_test.columns=['token','class']
df_test['token'] = df_test['token'].str.lower()


prior=dict()
##prior prob on 'O'
countOthers = len(df_train[df_train['class']=='O'])
prior['O'] = countOthers/totalDocs

#calculating the prior prob of class= MAJOR-CLAIM
countBMajor = len(df_train[df_train['class'] == 'B-MAJOR-CLAIM'])
countIMajor = len(df_train[df_train['class'] == 'I-MAJOR-CLAIM'])
prior['I-MAJOR-CLAIM'] = (countIMajor)/totalDocs
prior['B-MAJOR-CLAIM'] = (countBMajor)/totalDocs
#calculating the prior prob of class= Premise

countBPremise = len(df_train[df_train['class'] == 'B-PREMISE'])
countIPremise = len(df_train[df_train['class'] == 'I-PREMISE'])
prior['I-PREMISE'] = (countIPremise)/totalDocs
prior['B-PREMISE'] = (countBPremise)/totalDocs
#calculating the prior prob of class= CLAIM
countBClaim = len(df_train[df_train['class'] == 'B-CLAIM'])
countIClaim = len(df_train[df_train['class'] == 'I-CLAIM'])
prior['I-CLAIM'] = (countIClaim)/totalDocs
prior['B-CLAIM'] = (countBClaim)/totalDocs



posteriorProb = dict()
listTokens = df_test['token'].values.tolist()
classes = ['I-CLAIM', 'O', 'I-MAJOR-CLAIM', 'I-PREMISE','B-CLAIM','B-PREMISE','B-MAJOR-CLAIM']


# this method would return posterior prob of each word given the class for each class
def posterProb1(token):
    max = 0
    maxCl = ''
    totalTokenFreq = len(df_train[df_train['token'] == token])
    if totalTokenFreq == 0:
        return 'O'
    for cl in classes:
        probability = countClassToken(token, cl) * prior[cl] / totalTokenFreq
        if probability > max:
            max = probability
            maxCl = cl
    posteriorProb[token] = maxCl
    return maxCl





def countClassToken(token, classe):
    return len(df_train[(df_train['token'] == token) & (df_train['class'] == classe)])



i=0
flag=True

setTokens = set(listTokens)
print('testing is done on ',len(setTokens))
for token in setTokens:
    posteriorProb[token] = posterProb1(token)
print('training is finished')




testing is done on  3480
training is finished


In [157]:


dfTestOrig = pd.read_csv('./testBIO.txt',sep='\t',header=None)
dfTestOrig.columns = ['tokens','class']
listOriTest = dfTestOrig['tokens'].values.tolist()

# fileS=open('./results.txt','w')
fileS = open('./resultsNew.txt', 'w')
for token in listOriTest:
    #     print(token.lower())
    classLabel = ''
    if (token != token):
        print(' ', '\t', 'O', file=fileS)
        continue
        
    elif token in ('__END_PARAGRAPH__'):# '.', '?'):
#         flag = True
#         if token == '.':
        classLabel = 'O'
    else:
        classLabel = posteriorProb[token.lower()]


#         '+classLabel)
    print(str(token) + '\t' + classLabel, file=fileS)
fileS.close()


